# L-Type Calcium Channel

LTCC data broken out by document and assay for inspection , _e.g._ for comparison with the original publication.

See the notebook '[LTCC](LTCC.ipynb)' for various analyses of this data, and some more background information.

In [1]:
# pActivity column to use...

activity_col = 'pchembl_value'

## Load data

See notebook '[Data](Data.ipynb)' for further details.

In [2]:
# Load curve data (IC50, Ki, Kd etc)...

data = pd.read_pickle('data.pkl')

data.shape

(1317, 30)

In [3]:
HTML(data.head(2).to_html())

,target_chemblid,pref_name,target_type,organism,species,tissue,relationship_type,assay_chemblid,description,assay_organism,parent_cmpd_chemblid,compound_class,standard_type,standard_relation,standard_value,standard_units,pchembl_value,activity_comment,data_validity_comment,potential_duplicate,cmpd_chemblid,compound_key,published_type,published_relation,published_value,published_units,doc_chemblid,pubmed_id,reference,active
0,CHEMBL1940,Voltage-gated L-type calcium channel alpha-1C subunit,SINGLE PROTEIN,Homo sapiens,Cat,Heart,H,CHEMBL656260,Inhibition of (-)-[3H]- D-888 binding to L-type calcium channels in kitten heart ventricle membranes,Felis catus,CHEMBL138302,PAA,IC50,=,260,nM,6.59,None,None,NaN,CHEMBL138302,2d,IC50,=,0.26,uM,CHEMBL1127038,8474099,"J. Med. Chem., v. 36, p. 439 (1993)",1
1,CHEMBL1940,Voltage-gated L-type calcium channel alpha-1C subunit,SINGLE PROTEIN,Homo sapiens,Cat,Heart,H,CHEMBL656260,Inhibition of (-)-[3H]- D-888 binding to L-type calcium channels in kitten heart ventricle membranes,Felis catus,CHEMBL138302,PAA,IC50,=,390,nM,6.41,None,None,NaN,CHEMBL138302,2a,IC50,=,0.39,uM,CHEMBL1127038,8474099,"J. Med. Chem., v. 36, p. 439 (1993)",1


In [4]:
# Number of data records and number of unique parent compounds...

data.shape[0], data['parent_cmpd_chemblid'].drop_duplicates().size 

(1317, 897)

## Counts by document

Add counts of active and all parent compounds by document and add bibliographic information on documents; see notebook '[Docs](Docs.ipynb)' for further details.

In [5]:
# Get counts...

counts = data[['doc_chemblid', 'assay_chemblid']].drop_duplicates().groupby('doc_chemblid').apply(len).to_frame('n_assays')\
    .merge(data.groupby('doc_chemblid').apply(len).to_frame('n_total'), how='left', left_index=True, right_index=True)\
    .merge(data.query("active == 1").groupby('doc_chemblid').apply(len).to_frame('n_active'), how='left', left_index=True, right_index=True)\
    .fillna(0)\
    .sort(['n_total', 'n_active'], ascending=[False, False])\
    .reset_index()

In [6]:
# Add bibliographic information for documents..

counts = counts.merge(pd.read_pickle('docs.pkl'), how='left', left_on='doc_chemblid', right_index=True)

In [7]:
HTML(counts.head(2).to_html())

,doc_chemblid,n_assays,n_total,n_active,pubmed_id,title,abstract,citation,doi,links
0,CHEMBL2380260,2,180,79,23651412,"Structure-activity relationship of N,N'-disubstituted pyrimidinetriones as Ca(V)1.3 calcium channel-selective antagonists for Parkinson's disease.","CaV1.3 L-type calcium channels (LTCCs) have been a potential target for Parkinson's disease since calcium ion influx through the channel was implicated in the generation of mitochondrial oxidative stress, causing cell death in the dopaminergic neurons. Selective inhibition of CaV1.3 over other LTCC isoforms, especially CaV1.2, is critical to minimize potential side effects. We recently identified pyrimidinetriones (PYTs) as a CaV1.3-selective scaffold; here we report the structure-activity relationship of PYTs with both CaV1.3 and CaV1.2 LTCCs. By variation of the substituents on the cyclopentyl and arylalkyl groups of PYT, SAR studies allowed characterization of the CaV1.3 and CaV1.2 LTCCs binding sites. The SAR also identified four important moieties that either retain selectivity or enhance binding affinity. Our study represents a significant enhancement of the SAR of PYTs at CaV1.3 and CaV1.2 LTCCs and highlights several advances in the lead optimization and diversification of this family of compounds for drug development.","J. Med. Chem., v. 56, p. 4786 (2013)",,"Link_1 <a target=""_blank"" href=""http://wtgcsfx.hosted.exlibrisgroup.com/wtsc/cgi/public/docdel.cgi?source=Entrez%3APubMed+(Via+SFX)&meduid=23651412&month=6&author=Kang%2C+S&issue=11&ISBN=&article=Structure-activity+relationship+of+N%2CN'-disubstituted+pyrimidinetriones+as+Ca(V)1.3+calcium+channel-selective+antagonists+for+Parkinson's+disease.&publisher=American+Chemical+Society&journal=Journal+of+medicinal+chemistry&ISSN=0022-2623&abbrev=J+MED+CHEM&recipient=libadmin%40sanger.ac.uk&volume=56&genre=article&publiPlace=%5BEaston%2C+Pa.%5D&sender=&pages=4786-97&year=2013"">Link_2"
1,CHEMBL1158565,2,138,133,20382537,"Antagonism of 4-substituted 1,4-dihydropyridine-3,5-dicarboxylates toward voltage-dependent L-type Ca2+ channels Ca V 1.3 and Ca V 1.2.","L-type Ca(2+) channels in mammalian brain neurons have either a Ca(V)1.2 or Ca(V)1.3 pore-forming subunit. Recently, it was shown that Ca(V)1.3 Ca(2+) channels underlie autonomous pacemaking in adult dopaminergic neurons in the substantia nigra pars compacta, and this reliance renders them sensitive to toxins used to create animal models of Parkinson's disease. Antagonism of these channels with the dihydropyridine antihypertensive drug isradipine diminishes the reliance on Ca(2+) and the sensitivity of these neurons to toxins, pointing to a potential neuroprotective strategy. However, for neuroprotection without an antihypertensive side effect, selective Ca(V)1.3 channel antagonists are required. In an attempt to identify potent and selective antagonists of Ca(V)1.3 channels, 124 dihydropyridines (4-substituted-1,4-dihydropyridine-3,5-dicarboxylic diesters) were synthesized. The antagonism of heterologously expressed Ca(V)1.2 and Ca(V)1.3 channels was then tested using electrophysiological approaches and the FLIPR Calcium 4 assay. Despite the large diversity in substitution on the dihydropyridine scaffold, the most Ca(V)1.3 selectivity was only about twofold. These results support a highly similar dihydropyridine binding site at both Ca(V)1.2 and Ca(V)1.3 channels and suggests that other classes of compounds need to be identified for Ca(V)1.3 selectivity.","Bioorg. Med. Chem., v. 18, p. 3147 (2010)",,"Link_1 <a target=""_blank"" href=""http://wtgcsfx.hosted.exlibrisgroup.com/wtsc/cgi/public/docdel.cgi?source=Entrez%3APubMed+(Via+SFX)&meduid=20382537&month=5&author=Chang%2C+C+C&issue=9&ISBN=&article=Antagonism+of+4-substituted+1%2C4-dihydropyridine-3%2C5-dicarboxylates+toward+voltage-dependent+L-type+Ca2%2B+channels+Ca+V+1.3+and+Ca+V+1.2.&publisher=Pergamon&journal=Bioorganic+%26+medicinal+chemistry&ISSN=0968-0896&abbrev=BIOORG+MED+CHEM&recipient=libadmin

In [8]:
counts.to_pickle('counts.pkl')

## Main HTML report

Generate per-document report.

In [9]:
# Assay columns...

assay_cols = ['assay_chemblid', 'species', 'pref_name', 'tissue', 'description']

# A column subset for more convenient viewing...

cols = ['parent_cmpd_chemblid', 'compound_class', 'compound_key', 'active', 'pchembl_value', 'published_type', 'published_relation', 'published_value', 'published_units', 'activity_comment', 'data_validity_comment']

In [10]:
html_report = '<h2>Data by Document</h2>'

ordering = [] # Used in summary table

n_total = counts.shape[0]

for n, doc_chemblid in enumerate(counts['doc_chemblid'].tolist(), 1):
    
    # Get counts for assays for document...

    data_for_doc = data.query("doc_chemblid == @doc_chemblid")

    counts_for_doc = data_for_doc[assay_cols].drop_duplicates()\
        .merge(data.groupby('assay_chemblid').apply(len).to_frame('n_total'), how='left', left_on='assay_chemblid', right_index=True)\
        .merge(data.query("active == 1").groupby('assay_chemblid').apply(len).to_frame('n_active'), how='left', left_on='assay_chemblid', right_index=True)\
        .fillna(0)\
        .sort(['n_total', 'n_active'], ascending=[False, False])\
        .reset_index(drop=True)

    # Add links to data tables for assays (NB making copy)...
    
    counts_for_doc2 = counts_for_doc.copy()
    
    counts_for_doc2['assay_chemblid'] = counts_for_doc2['assay_chemblid'].apply(lambda x: '<a target="_blank" href="#{}_1">{}</a>'.format(* [x]*2))

    #  HTML for document...

    html_report += '<a name="{}_1"></a> <h3><a target="_blank" href="https://www.ebi.ac.uk/chembl/doc/inspect/{}">{}</a> [<a href="#{}_0">*</a>] ({}/{})</h3>'.format(* [doc_chemblid]*4 + [n, n_total]) \
        + counts.query("doc_chemblid == @doc_chemblid").reset_index(drop=True).iloc[:, 1:].to_html() \
        + counts_for_doc2.to_html()

    # HTML for assays for document...

    m_total = counts_for_doc.shape[0]

    for m, assay_chemblid in enumerate(counts_for_doc['assay_chemblid'].tolist(), 1):

        data_for_assay = data.query('assay_chemblid == @assay_chemblid').sort(['active', activity_col], ascending=[0, 0]).reset_index(drop=True)

        data_for_assay['img'] = data_for_assay['parent_cmpd_chemblid'].apply(lambda x: '<img src="https://www.ebi.ac.uk/chembl/api/data/image/{}.svg" height="400" width="400"> <a target="_blank" href="https://www.ebi.ac.uk/chembl/compound/inspect/{}">{}</a>'.format(*(x,)*3))

        data_for_assay = data_for_assay[[data_for_assay.columns.tolist()[-1]] + data_for_assay.columns.tolist()[:-1]] # Put image at left-hand side of table

        html_report += '<a name="{}_1"></a> <h4><a target="_blank" href="https://www.ebi.ac.uk/chembl/assay/inspect/{}">{}</a> [<a href="#{}_0">*</a>] [{}/{}]</h4>'.format(* [assay_chemblid]*4 + [m, m_total]) \
            + counts_for_doc.query("assay_chemblid == @assay_chemblid").iloc[:, 1:].to_html() \
            + data_for_assay[['img'] + cols].to_html()
            
        ordering.append((doc_chemblid, assay_chemblid))

### HTML summary table and spreadsheet

In [11]:
# Generate summary table...
# NB The ordering from the main report is used here (as the assays are ordered by size within their target, which would be fiddly to do on the summary table), hence this occurs after the main report is generated.

summary = data[['doc_chemblid'] + assay_cols].drop_duplicates()\
    .merge(data[['doc_chemblid', 'assay_chemblid']].drop_duplicates().groupby('doc_chemblid').apply(len).to_frame('n_assays'), how='left', left_on='doc_chemblid', right_index=True)\
    .merge(data.groupby('doc_chemblid').apply(len).to_frame('n_total_doc'), how='left', left_on='doc_chemblid', right_index=True)\
    .merge(data.query("active == 1").groupby('doc_chemblid').apply(len).to_frame('n_active_doc'), how='left', left_on='doc_chemblid', right_index=True)\
    .merge(data.groupby('assay_chemblid').apply(len).to_frame('n_total_assay'), how='left', left_on='assay_chemblid', right_index=True)\
    .merge(data.query("active == 1").groupby('assay_chemblid').apply(len).to_frame('n_active_assay'), how='left', left_on='assay_chemblid', right_index=True)\
    .fillna(0)\
    .set_index(['doc_chemblid', 'assay_chemblid'])\
    .loc[pd.DataFrame(ordering, columns=['doc_chemblid', 'assay_chemblid']).set_index(['doc_chemblid', 'assay_chemblid']).index.values]\
    .reset_index()

# Add index numbers for targets and assays (same as in main report)...

n = pd.DataFrame(((x, i) for i, x in enumerate(summary['doc_chemblid'].drop_duplicates(), 1)), columns=['doc_chemblid', 'n'])

m = summary.groupby('doc_chemblid').apply(lambda group: pd.DataFrame(((x, i) for i, x in enumerate(group['assay_chemblid'], 1)), columns=['assay_chemblid', 'm']).reset_index(drop=True).set_index('assay_chemblid')).reset_index()

nm = n.merge(m, on='doc_chemblid')

summary = summary.merge(nm, left_on=['doc_chemblid', 'assay_chemblid'], right_on=['doc_chemblid', 'assay_chemblid'])

summary = summary[['n', 'doc_chemblid', 'n_total_doc', 'n_active_doc', 'n_assays', 'm'] + assay_cols + ['n_total_assay', 'n_active_assay']]

# Take copy before adding links...

summary1 = summary.copy()

# Add links to main report (using copy)...

for col in ['doc_chemblid', 'assay_chemblid']: summary1[col] = summary1[col].apply(lambda x: '<a name="{}_0"> </a><a href="#{}_1">{}</a>'.format(*(x,)*3))
    
# Prepend to HTML...

html_report = '<h2>Document Summary</h2>' + summary1.to_html() + html_report

In [12]:
# Display HTML...

HTML(html_report)

,n,doc_chemblid,n_total_doc,n_active_doc,n_assays,m,assay_chemblid,species,pref_name,tissue,description,n_total_assay,n_active_assay
0,1,CHEMBL2380260,180,79,2,1,CHEMBL2384388,Rat,Voltage-gated L-type calcium channel alpha-1D subunit,,Antagonist activity at rat CaV1.3alpha1D transfected in HEK293 cells by FLIPR assay,90,52
1,1,CHEMBL2380260,180,79,2,2,CHEMBL2384387,Rabbit,Voltage-gated L-type calcium channel alpha-1C subunit,,Antagonist activity at rabbit CaV1.2alpha1C transfected in HEK293 cells by FLIPR assay,90,27
2,2,CHEMBL1158565,138,133,2,1,CHEMBL1121084,Rat,Voltage-gated L-type calcium channel alpha-1D subunit,,Antagonist activity at rat Cav1.3 expressed in HEK293 cells assessed as inhibition of voltage pulse-induced calcium current by FLIPR calcium 4 assay,69,67
3,2,CHEMBL1158565,138,133,2,2,CHEMBL1121085,Rabbit,Voltage-gated L-type calcium channel alpha-1C subunit,,Antagonist activity at rabbit Cav1.2 expressed in HEK293 cells assessed as inhibition of voltage pulse-induced calcium current by FLIPR calcium 4 assay,69,66
4,3,CHEMBL1125592,93,52,1,1,CHEMBL664572,Rat,Voltage-gated L-type calcium channel,,Affinity to bind dihydropyridine receptor (DHP receptor) by inhibiting the radioligand [3H]nitrendipine,93,52
5,4,CHEMBL2396637,74,50,2,1,CHEMBL2399368,Rabbit,Voltage-gated L-type calcium channel alpha-1C subunit,,Antagonist activity at rabbit voltage-dependent L-type calcium channel Cav1.2alpha-1C expressed in HEK293 cells by FLIPR assay,37,26
6,4,CHEMBL2396637,74,50,2,2,CHEMBL2399369,Rat,Voltage-gated L-type calcium channel alpha-1D subunit,,Antagonist activity at rat voltage-dependent L-type calcium channel Cav1.3alpha-1D expressed in HEK293 cells by FLIPR assay,37,24
7,5,CHEMBL1128175,71,63,5,1,CHEMBL704173,Rat,Voltage-gated L-type calcium channel,Brain,Inhibition of [3H]nitrendipine binding at L-type [Ca2+] channel in rat cortex homogenate by 50%.,35,28
8,5,CHEMBL1128175,71,63,5,2,CHEMBL703969,Rat,Voltage-gated L-type calcium channel,Brain,Calcium antagonistic activity by measuring [3H]nitrendipine displacement at L-type [Ca2+] channel in rat cortex homogenate,17,17
9,5,CHEMBL1128175,71,63,5,3,CHEMBL703971,Rat,Voltage-gated L-type calcium channel,Brain,Calcium antagonistic activity by measuring [3H]nitrendipine displacement at calcium channel in rat cortex homogenate,11,11


In [13]:
# Save HTML...

open('LTCC_Report.html', 'w').write(html_report)

In [14]:
# Save as Excel Workbook...

summary.to_excel('summary.xlsx', index=False)